<a href="https://colab.research.google.com/github/simon-clematide/colab-notebooks-for-teaching/blob/main/openai-keyphrase-starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Keyphrase extraction sample code

This notebook:
  - Defines the input data inline as a JSON string (no file loading)
  - Extracts exactly three English keyphrases per document
  -	Writes the results to:
    - a JSONL file (canonical output)
    - an Excel file with DOCID, full text, and three keyphrase columns for human inspection and validation

## Setup

In [ ]:
%pip install openai openpyxl

In [ ]:
import os
import json
from pathlib import Path
from typing import List

from openpyxl import Workbook
from openai import OpenAI

# --- OpenAI API key handling ---

# Option 1: temporary manual paste (DO NOT commit or publish)
API_KEY = None  # e.g. "sk-..."  (leave as None in shared code)

# Option 2: environment variable / Colab secrets (recommended)
try:
    # Available in Google Colab
    from google.colab import userdata
    API_KEY = API_KEY or os.environ.get("OPENAI_API_KEY") or userdata.get("OPENAI_API_KEY")
except ImportError:
    # Local Jupyter / script
    API_KEY = API_KEY or os.environ.get("OPENAI_API_KEY")

if not API_KEY:
    raise RuntimeError(
        "OPENAI_API_KEY not found. "
        "Set it as an environment variable or via Colab userdata."
    )

client = OpenAI(api_key=API_KEY)

## Input data (JSON)
Historical news documents

In [ ]:
DATA_JSON = """
[
  {
    "doc_id": "DOC_001",
    "text": "Rom. 10. April. Das Luftschiff \\"Graf Zeppelin\\" passierte auf seiner Nrgyptensahrt gegen 20 Uhr Mez. die Westspitze non Sizilien und schlug non dort aus südöstlichen Kurs ein."
  },
  {
    "doc_id": "DOC_002",
    "text": "St-Pétersbourg, 31. Pendant les dernières 24 heures, 50 personnes sont tombées malades du choléra. Il y a iu 19 décès. Le nombre des malades s'élève actuellement à 651."
  },
  {
    "doc_id": "DOC_003",
    "text": "Schirru plante ein Attentat auf Mussolini. Rom, 6. gebr. Der Anarchist Schirru wurde von ber internationalen Polizei feit langer Zeit überwacht. Er ist aus Sardinien gebürtig und jetzt 32 Jahre alt. Vor mehreren Iay« ren wa« er nach Amerika ausgewandert unb hatte sich bort naturalisiert, so baß ihm jetzt ein amerikanischer Paß bas Reisen «rleichter:? unb weniger Aufsehen erregte. Seine gamilie war ihm nach Amerika nachgefolgt. Im März 1930 verließ er Amerika wieder."
  },
  {
    "doc_id": "DOC_004",
    "text": "Der Königsbesuch im Vatikan. Rom, 5. Dez. (Europapreß). Im endgültigen Programm für den heutigen Befuch des italienischen Königspaares im Vatikan ist der geistliche Charakter gegenüber bem politischen in den Vordergrund gerückt worden. Obwohl der Besuch erst um 11 Uhr stattfindet, sind die Zugänge der Vatikanstadt schon seit den ersten Morgenstunben gesperrt unb umfassende Sicherheitsmaßnahmen im Vatikan selbst getroffen worben."
  },
  {
    "doc_id": "DOC_005",
    "text": "Jugoslawien protestiert in Rom. Budapest, 29. Mai. (Wolff.) Der jugostawifche gesandte in Rom hat bei dem italienischen Staatssekretär Grandi wegen der in italienischen Städten vorgekommenen Zwischenfälle schriftliche Vorstellungen erhoben. In Zara hatten fafzistifche Demonstranten jugoslawische Gefchustsläden geplündert und den jugoflawifchen Konful Simitfch tätlich angegriffen. Ferner habe man in mehreren italienifchen Städten die serbische Fahne heruntergerissen und das Bild des Königs Alexander verfetzt. Die jugostawifche Regierung könne diese Demonstrattonen nicht ohne weiteres hinnehmen und sei geneigt, Genugtuung zu fordern. Grandi teilte mit, die italienische Regierung werden schriftlich antworten."
  },
  {
    "doc_id": "DOC_006",
    "text": "Die Diebe i» deutschen Botschafterpalais in Rom. ««Land, 6. März. Der \\"Cornere della Sera\\" meldet WM Einbruch in die deutfche Botschaft in Rom: Die Verhafteten wurden von der Polizei einem eingehenden Verhör unterworfen, über das aber strengstens Stillschweigen bewahrt uirb. 3mnerhin erfahrt man, dafz dic Einbrttyer erklärten, sie hätten sich nur vorübergehend in Ar» «usgehcilien. Sie weigerten f.ch die Ziamen ihrer tswfAiitn anzugeben, unb behaupteten, sie seien nur in die Botschaft eingedrungen, um Geld zu stehlen. Sie hätten auf eigene Rechnung gehandelt, ohne jede Beeinßllssnng durch Vertreter einer fremden Macht. Auch dementiert man, daß der eine Einbrecher Ofn^ier fei."
  },
  {
    "doc_id": "DOC_007",
    "text": "«nkunft des päpstlichen Nuntius in Berlin. Berlin, 25. April. (Wolff). Der neue päpstliche Nuntius für Berlin, Eefare Orfenigo, traf heute vormittag 8.50 Uhr von Rom auf dem Potsdamer Bahnhof ein. Zu feiner Begrüßung hatten sich im Auftrage der Reichsregterung der Ehef des Protokolls Graf Tattenberg und der Vatikanreferet Legationsrat Dr. Klee eingefunden, welter der Berliner Bischof Dr. Schreiber, der Geschäftstra ger des Hl. Stuhls, Mgr. Eontoz. Nuntiaturrat Dr. Vanasch; für die katholische Aktion war der Vorsitzende Ministerialdirelttor Dr. Klausener, Stadtbaurat Dr. Adler und eine Anzahl weiterer Vorstandsmitglieder und Vertreter des Berliner Klerus erfäiienen. Der Nuntius erwiderte auf die Begrühungsanspräche von Graf Battenberg in geläufigem Deutsch."
  },
  {
    "doc_id": "DOC_008",
    "text": "Verlobung des bulgarischen Königs mil einer italienischen Prinzessin. Rom, 4. Okt. ag. (Stefani.) Der König hat seine Zustimmung zu der Verlobung seiner zweitjüngsten Tochter, der 190? geborenen Prinzessin Giovanna, mit dem König Boris von Bulgarien gegeben."
  }
]
"""

documents = json.loads(DATA_JSON)

## Settings for processing

In [ ]:
MODEL_NAME = "gpt-4o-mini" # gpt-4  gpt-4.1
N_KEYPHRASES = 3

## Generative AI Prompting

In [ ]:
def extract_three_keyphrases(text: str) -> List[str]:
    """
    Extract exactly three concise English keyphrases from a historical news text.

    The function performs no preprocessing or normalization. It relies entirely
    on the model instruction to return valid JSON.

    Args:
        text (str): The input news text (raw OCR, multilingual allowed).

    Returns:
        List[str]: A list of exactly three English keyphrases.

    Raises:
        KeyError: If the expected JSON schema is not returned.
        json.JSONDecodeError: If the model output is not valid JSON.
    """

    user_prompt = f"""
Extract exactly THREE concise English keyphrases that best categorize
the historical news text below.

Return ONLY valid JSON with the following schema:

{{"keyphrases": [string, string, string]}}

Text:
{text}
"""

    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {
                "role": "system",
                "content": (
                    """You extract topical keyphrases from historical news texts.
                    Keyphrases must be concise and in English."""
                ),
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
        response_format={"type": "json_object"},
        temperature=1,
        max_tokens=200,
    )

    data = json.loads(response.choices[0].message.content)
    keyphrases = data["keyphrases"]

    # Defensive sanity check (no correction, only validation)
    if len(keyphrases) != N_KEYPHRASES:
        raise ValueError(f"Expected {N_KEYPHRASES} keyphrases, got {len(keyphrases)}")

    return keyphrases

## Prepare the output files

In [ ]:
jsonl_path = Path("keyphrases.jsonl")
excel_path = Path("keyphrases.xlsx")

workbook = Workbook()
worksheet = workbook.active
worksheet.title = "Evaluation"

worksheet.append(
    [
        "DOC_ID",
        "Text",
        "Keyphrase 1",
        "Keyphrase 2",
        "Keyphrase 3",
    ]
)

## Process all documents

In [ ]:
with jsonl_path.open("w", encoding="utf-8") as jsonl_file:
    for doc in documents:
        doc_id = doc["doc_id"]
        print(f"processing {doc_id}")
        text = doc["text"]

        keyphrases = extract_three_keyphrases(text)

        # --- JSONL (canonical output) ---
        record = {
            "doc_id": doc_id,
            "text": text,
            "keyphrases": keyphrases,
        }
        jsonl_file.write(json.dumps(record, ensure_ascii=False) + "\n")

        # --- Excel (human inspection) ---
        worksheet.append(
            [
                doc_id,
                text,
                keyphrases[0],
                keyphrases[1],
                keyphrases[2],
            ]
        )

In [ ]:
worksheet.column_dimensions["A"].width = 12
worksheet.column_dimensions["B"].width = 90
worksheet.column_dimensions["C"].width = 30
worksheet.column_dimensions["D"].width = 30
worksheet.column_dimensions["E"].width = 30

workbook.save(excel_path)

Run the sample code. Then modify the prompt to your data.